In [1]:
import tensorflow as tf
import numpy as np

In [2]:
a=tf.Variable([[True,False],[False,True]],dtype=bool)

In [5]:
b=tf.Variable([[3,4],[1,2]],dtype=tf.float32)

In [12]:
tf.train.exponential_decay

0.9320653479069899

In [ ]:
tf.nn.dropout

In [22]:
**60

0.012852183368996866

In [20]:
np.random.randn(4,2,2)-np.random.randn(4,2,2).mean((1,2))

ValueError: operands could not be broadcast together with shapes (4,2,2) (4,) 